In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

In [2]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [3]:
file_name = "/media/ssd/test/standardized-datasets/netflow/nf_unsw_nb15_standardized.csv"
data = pd.read_csv(file_name)

In [4]:
data.Label.value_counts()

Label
0    2295222
1      95053
Name: count, dtype: int64

In [5]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [6]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [7]:
data.Attack.unique()

array(['Benign', 'Exploits', 'Generic', 'Fuzzers', 'Backdoor', 'DoS',
       'Reconnaissance', 'Shellcode', 'Worms', 'Analysis'], dtype=object)

In [8]:
data = data.groupby(by='Attack').sample(frac=0.1, random_state=13)

In [9]:
data.groupby(by="Attack").count()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label,flow_id,dataset_source
Attack,,,,,,,,,,,,,,,,,,,,,
Analysis,230,230,230,230,230,230,230,230,230,230,...,230,230,230,230,230,230,230,230,230,230
Backdoor,217,217,217,217,217,217,217,217,217,217,...,217,217,217,217,217,217,217,217,217,217
Benign,229522,229522,229522,229522,229522,229522,229522,229522,229522,229522,...,229522,229522,229522,229522,229522,229522,229522,229522,229522,229522
DoS,579,579,579,579,579,579,579,579,579,579,...,579,579,579,579,579,579,579,579,579,579
Exploits,3155,3155,3155,3155,3155,3155,3155,3155,3155,3155,...,3155,3155,3155,3155,3155,3155,3155,3155,3155,3155
Fuzzers,2231,2231,2231,2231,2231,2231,2231,2231,2231,2231,...,2231,2231,2231,2231,2231,2231,2231,2231,2231,2231
Generic,1656,1656,1656,1656,1656,1656,1656,1656,1656,1656,...,1656,1656,1656,1656,1656,1656,1656,1656,1656,1656
Reconnaissance,1278,1278,1278,1278,1278,1278,1278,1278,1278,1278,...,1278,1278,1278,1278,1278,1278,1278,1278,1278,1278
Shellcode,143,143,143,143,143,143,143,143,143,143,...,143,143,143,143,143,143,143,143,143,143


In [10]:
X = data.drop(columns=["Attack", "Label"])
y = data[["Attack", "Label"]]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)

In [11]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [12]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [13]:
if 'dataset_source' in X_train.columns:
    X_train = X_train.drop(columns=['dataset_source'])
if 'dataset_source' in X_test.columns:
    X_test = X_test.drop(columns=['dataset_source'])

In [14]:
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [15]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,flow_id,h
220614,59.166.0.4,149.171.126.1,4.799343e-09,3.046911e-09,3.992291e-05,7.648066e-07,0.000072,1.070729e-06,1.979102e-09,1.979232e-09,...,1.328928e-03,1.328928e-03,4.041250e-10,4.041250e-10,6.310571e-09,6.310616e-09,0.000000,3.616402e-09,0.033745,"[4.79934325210543e-09, 3.046911229391236e-09, ..."
329844,59.166.0.1,149.171.126.8,9.004294e-09,8.034034e-09,1.076602e-04,8.281552e-07,0.000063,8.281552e-07,1.393801e-08,1.393694e-08,...,0.000000e+00,0.000000e+00,1.179036e-08,1.179838e-08,8.541591e-09,8.541651e-09,0.000000,9.249878e-09,0.068291,"[9.004293599058882e-09, 8.0340336076954e-09, 0..."
1093665,59.166.0.3,149.171.126.8,3.068685e-13,8.437668e-13,1.638983e-07,3.061234e-09,0.000010,7.129838e-09,1.188386e-13,1.274923e-13,...,6.372843e-07,7.080936e-08,4.545956e-14,5.191930e-14,4.034959e-13,4.034988e-13,0.000000,4.369547e-13,0.000011,"[3.068684887458359e-13, 8.437667728375511e-13,..."
1764268,59.166.0.4,149.171.126.0,1.705243e-08,1.521494e-08,5.724548e-05,7.841846e-07,0.000070,7.841846e-07,2.639595e-08,2.639392e-08,...,0.000000e+00,0.000000e+00,2.232872e-08,2.234389e-08,1.356214e-08,2.583379e-10,0.000024,1.751752e-08,0.691756,"[1.7052427913141602e-08, 1.5214939122076362e-0..."
1525176,59.166.0.9,149.171.126.5,2.309217e-10,2.855922e-10,1.717771e-05,2.649519e-07,0.000124,2.796714e-07,8.942727e-11,9.593926e-11,...,1.598543e-04,1.065695e-04,5.174558e-12,5.174558e-12,3.036348e-10,3.036369e-10,0.000000,3.288128e-10,0.011225,"[2.3092165458478978e-10, 2.855922428474074e-10..."


In [16]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [17]:
# Training graph

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack"], create_using=nx.MultiGraph())

train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [18]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [19]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [20]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [21]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [22]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [23]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to(device)

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [24]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [25]:
# Convert to GPU
train_g = train_g.to(device)

In [26]:
cnt_wait = 0
best = 1e9
best_t = 0
dur = []
node_features = train_g.ndata['h']
edge_features = train_g.edata['h']

for epoch in range(epochs):
    dgi.train()
    if epoch >= 3:
        t0 = time.time()

    dgi_optimizer.zero_grad()
    loss = dgi(train_g, node_features, edge_features)
    loss.backward()
    dgi_optimizer.step()

    if loss < best:
        best = loss
        best_t = epoch
        cnt_wait = 0
        torch.save(dgi.state_dict(), 'best_dgi.pkl')
    else:
        cnt_wait += 1

  # if cnt_wait == patience:
  #     print('Early stopping!')
  #     break

    if epoch >= 3:
        dur.append(time.time() - t0)

    if epoch % 50 == 0:

        print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
            "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur),
              loss.item(),
              train_g.num_edges() / np.mean(dur) / 1000))
# ... existing code ...

/media/ssd/test/gnn_cuda_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/media/ssd/test/gnn_cuda_env/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Time(s) nan | Loss 1.9820 | ETputs(KTEPS) nan
Epoch 00050 | Time(s) 0.0790 | Loss 1.4002 | ETputs(KTEPS) 4237.33
Epoch 00100 | Time(s) 0.0794 | Loss 1.3846 | ETputs(KTEPS) 4214.57
Epoch 00150 | Time(s) 0.0800 | Loss 1.3821 | ETputs(KTEPS) 4184.01
Epoch 00200 | Time(s) 0.0800 | Loss 1.3789 | ETputs(KTEPS) 4181.20
Epoch 00250 | Time(s) 0.0801 | Loss 1.3649 | ETputs(KTEPS) 4179.90
Epoch 00300 | Time(s) 0.0801 | Loss 1.3529 | ETputs(KTEPS) 4178.61
Epoch 00350 | Time(s) 0.0800 | Loss 1.3231 | ETputs(KTEPS) 4180.89
Epoch 00400 | Time(s) 0.0800 | Loss 1.2865 | ETputs(KTEPS) 4183.42
Epoch 00450 | Time(s) 0.0800 | Loss 1.2024 | ETputs(KTEPS) 4182.78
Epoch 00500 | Time(s) 0.0800 | Loss 1.0599 | ETputs(KTEPS) 4185.48
Epoch 00550 | Time(s) 0.0798 | Loss 1.3756 | ETputs(KTEPS) 4191.48
Epoch 00600 | Time(s) 0.0797 | Loss 0.9476 | ETputs(KTEPS) 4199.84
Epoch 00650 | Time(s) 0.0796 | Loss 0.7150 | ETputs(KTEPS) 4203.58
Epoch 00700 | Time(s) 0.0795 | Loss 0.6629 | ETputs(KTEPS) 4207.29
Ep

In [27]:
dgi.load_state_dict(torch.load('best_dgi.pkl'))

/tmp/ipykernel_4188052/597080661.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dgi.load_state_dict(torch.load('best_dgi.pkl'))


<All keys matched successfully>

In [28]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [29]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [30]:
# Convert to GPU
test_g = test_g.to(device)

In [31]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [32]:
df_train = pd.DataFrame(training_emb, )
df_train["Attack"] = lab_enc.inverse_transform(
        train_g.edata['Attack'].detach().cpu().numpy())
df_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb, )
df_test["Attack"] = lab_enc.inverse_transform(
        test_g.edata['Attack'].detach().cpu().numpy())
df_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

In [33]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,Attack,Label
0,0.028651,0.01135,-0.016843,-0.010814,0.003143,0.003494,0.018754,0.016268,0.005725,-0.009464,...,0.041502,-0.021986,-0.010696,-0.001949,0.043754,-0.017725,-0.032184,0.001297,Benign,0
1,0.028651,0.01135,-0.016843,-0.010814,0.003143,0.003494,0.018754,0.016268,0.005725,-0.009464,...,0.041502,-0.021986,-0.010696,-0.001949,0.043754,-0.017725,-0.032184,0.001297,Benign,0
2,0.028651,0.01135,-0.016843,-0.010814,0.003143,0.003494,0.018754,0.016268,0.005725,-0.009464,...,0.041502,-0.021986,-0.010696,-0.001949,0.043754,-0.017725,-0.032184,0.001297,Benign,0
3,0.028651,0.01135,-0.016843,-0.010814,0.003143,0.003494,0.018754,0.016268,0.005725,-0.009464,...,0.041502,-0.021986,-0.010696,-0.001949,0.043754,-0.017725,-0.032184,0.001297,Benign,0
4,0.028651,0.01135,-0.016843,-0.010814,0.003143,0.003494,0.018754,0.016268,0.005725,-0.009464,...,0.041502,-0.021986,-0.010696,-0.001949,0.043754,-0.017725,-0.032184,0.001297,Benign,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334631,0.021734,0.23236,-0.001371,-0.342937,0.003244,-0.171518,-0.124611,0.002098,-0.117244,0.030774,...,0.820784,0.224955,0.137295,-0.250774,0.488516,-0.193601,-0.578108,-0.042180,Benign,0
334632,0.021734,0.23236,-0.001371,-0.342937,0.003244,-0.171518,-0.124611,0.002098,-0.117244,0.030774,...,0.820784,0.224955,0.137295,-0.250774,0.488516,-0.193601,-0.578108,-0.042180,Benign,0
334633,0.021734,0.23236,-0.001371,-0.342937,0.003244,-0.171518,-0.124611,0.002098,-0.117244,0.030774,...,0.820784,0.224955,0.137295,-0.250774,0.488516,-0.193601,-0.578108,-0.042180,Benign,0
334634,0.021734,0.23236,-0.001371,-0.342937,0.003244,-0.171518,-0.124611,0.002098,-0.117244,0.030774,...,0.820784,0.224955,0.137295,-0.250774,0.488516,-0.193601,-0.578108,-0.042180,Benign,0


# Embeddings CBLOF  Embeddings

In [34]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [35]:
benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label", "Attack"])
normal_train_samples = df_train.drop(columns=["Label", "Attack"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label", "Attack"])

In [36]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [37]:
%pip install pyod

Note: you may need to restart the kernel to use updated packages.


In [38]:
from pyod.models.cblof import CBLOF
n_est = [2,3,5,7,9,10]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]/media/ssd/test/gnn_cuda_env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
  3%|▎         | 1/36 [00:12<07:10, 12.29s/it]/media/ssd/test/gnn_cuda_env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
  6%|▌         | 2/36 [00:23<06:31, 11.52s/it]/media/ssd/test/gnn_cuda_env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
  8

{'n_estimators': 2, 'con': 0.01}
0.8125843406399277
              precision    recall  f1-score   support

           0     0.9823    0.9912    0.9867    137714
           1     0.7282    0.5684    0.6384      5704

    accuracy                         0.9744    143418
   macro avg     0.8552    0.7798    0.8126    143418
weighted avg     0.9722    0.9744    0.9729    143418



In [55]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]/media/ssd/test/gnn_cuda_env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
  3%|▎         | 1/36 [00:11<06:50, 11.74s/it]/media/ssd/test/gnn_cuda_env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
  6%|▌         | 2/36 [00:23<06:32, 11.55s/it]/media/ssd/test/gnn_cuda_env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
  8

In [ ]:
###  CBLOF RAW

In [56]:
df_raw_train = pd.concat([X_train.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_train], axis=1)
df_raw_test = pd.concat([X_test.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_test], axis=1)

In [57]:
raw_benign_train_samples = df_raw_train[df_raw_train.Label == 0].drop(columns=["Label", "Attack"])
raw_normal_train_samples = df_raw_train.drop(columns=["Label", "Attack"])

raw_train_labels = df_raw_train["Label"]
raw_test_labels = df_raw_test["Label"]

raw_test_samples = df_raw_test.drop(columns=["Label", "Attack"])

In [58]:
from pyod.models.cblof import CBLOF

n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = CBLOF(n_clusters=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]/media/ssd/test/gnn_cuda_env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/media/ssd/test/gnn_cuda_env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/media/ssd/test/gnn_cuda_env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/media/ssd/test/gnn_cuda_env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: Futur

2
2
2
2
2
2


 19%|█▉        | 7/36 [00:01<00:05,  5.27it/s]/media/ssd/test/gnn_cuda_env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
 22%|██▏       | 8/36 [00:01<00:05,  5.10it/s]/media/ssd/test/gnn_cuda_env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
 25%|██▌       | 9/36 [00:01<00:05,  5.02it/s]/media/ssd/test/gnn_cuda_env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init

{'n_estimators': 10, 'con': 0.2}
0.3971858513564739
              precision    recall  f1-score   support

           0     0.0294    0.7949    0.0566        39
           1     0.9945    0.5864    0.7378      2478

    accuracy                         0.5896      2517
   macro avg     0.5119    0.6906    0.3972      2517
weighted avg     0.9796    0.5896    0.7272      2517



In [ ]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [20:42<00:00, 34.51s/it]


benign only
{'n_estimators': 2}
0.8618432811826696
              precision    recall  f1-score   support

           0     0.9591    0.9806    0.9697    499068
           1     0.8287    0.6916    0.7540     67744

    accuracy                         0.9461    566812
   macro avg     0.8939    0.8361    0.8618    566812
weighted avg     0.9435    0.9461    0.9439    566812



In [ ]:
# HBOS  Embeddings

In [ ]:
benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label", "Attack"])
normal_train_samples = df_train.drop(columns=["Label", "Attack"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label", "Attack"])

In [ ]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [ ]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [19:29<00:00, 32.49s/it]


{'n_estimators': 5, 'con': 0.01}
0.945069359337394
              precision    recall  f1-score   support

           0     0.9845    0.9897    0.9871    996643
           1     0.9213    0.8855    0.9030    135488

    accuracy                         0.9772   1132131
   macro avg     0.9529    0.9376    0.9451   1132131
weighted avg     0.9769    0.9772    0.9770   1132131



In [ ]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [21:10<00:00, 35.28s/it]


{'n_estimators': 5, 'con': 0.1}
0.9189314026948445
              precision    recall  f1-score   support

           0     0.9705    0.9945    0.9824    996643
           1     0.9503    0.7779    0.8555    135488

    accuracy                         0.9686   1132131
   macro avg     0.9604    0.8862    0.9189   1132131
weighted avg     0.9681    0.9686    0.9672   1132131



In [ ]:
##  HBOS  RAw

In [ ]:
from pyod.models.cblof import CBLOF

n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = HBOS(n_bins=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [02:09<00:00,  3.59s/it]


{'n_estimators': 30, 'con': 0.04}
0.8627757960209628
              precision    recall  f1-score   support

           0     0.9715    0.9601    0.9658    499068
           1     0.7294    0.7928    0.7598     67744

    accuracy                         0.9401    566812
   macro avg     0.8505    0.8765    0.8628    566812
weighted avg     0.9426    0.9401    0.9412    566812



In [ ]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = HBOS(n_bins=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [02:14<00:00,  3.74s/it]


benign only
{'n_estimators': 5}
0.7882018992795334
              precision    recall  f1-score   support

           0     0.9766    0.8943    0.9337    499068
           1     0.5197    0.8422    0.6427     67744

    accuracy                         0.8881    566812
   macro avg     0.7481    0.8683    0.7882    566812
weighted avg     0.9220    0.8881    0.8989    566812



In [ ]:
##  PCA  Emb

In [ ]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [29:47<00:00, 49.66s/it]


{'n_estimators': 10, 'con': 0.001}
0.9442956641768174
              precision    recall  f1-score   support

           0     0.9770    0.9988    0.9878    996643
           1     0.9896    0.8267    0.9008    135488

    accuracy                         0.9782   1132131
   macro avg     0.9833    0.9127    0.9443   1132131
weighted avg     0.9785    0.9782    0.9774   1132131



In [ ]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [32:26<00:00, 54.07s/it]


{'n_estimators': 10, 'con': 0.1}
0.9256946497974641
              precision    recall  f1-score   support

           0     0.9723    0.9955    0.9838    996643
           1     0.9598    0.7916    0.8676    135488

    accuracy                         0.9711   1132131
   macro avg     0.9661    0.8935    0.9257   1132131
weighted avg     0.9708    0.9711    0.9699   1132131



In [ ]:
##  PCA  RAw

In [ ]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples)
   
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [07:28<00:00, 12.47s/it]


{'n_estimators': 20, 'con': 0.1}
0.7684270275042566
              precision    recall  f1-score   support

           0     0.9639    0.9009    0.9313    499068
           1     0.5071    0.7513    0.6055     67744

    accuracy                         0.8830    566812
   macro avg     0.7355    0.8261    0.7684    566812
weighted avg     0.9093    0.8830    0.8924    566812



In [ ]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples)

    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [08:04<00:00, 13.46s/it]


benign only
{'n_estimators': 10}
0.7376147683157477
              precision    recall  f1-score   support

           0     0.9622    0.8744    0.9162    499068
           1     0.4466    0.7471    0.5590     67744

    accuracy                         0.8591    566812
   macro avg     0.7044    0.8107    0.7376    566812
weighted avg     0.9006    0.8591    0.8735    566812



In [ ]:
##  IF  Emb

In [ ]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [3:09:47<00:00, 474.46s/it]  


{'n_estimators': 20, 'con': 0.001}
0.9538863735449246
              precision    recall  f1-score   support

           0     0.9805    0.9992    0.9897    996643
           1     0.9928    0.8538    0.9180    135488

    accuracy                         0.9818   1132131
   macro avg     0.9866    0.9265    0.9539   1132131
weighted avg     0.9820    0.9818    0.9812   1132131



In [ ]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [3:31:56<00:00, 529.86s/it]  


{'n_estimators': 50, 'con': 0.2}
0.8110535459623227
              precision    recall  f1-score   support

           0     0.9878    0.8952    0.9392    996643
           1     0.5436    0.9184    0.6829    135488

    accuracy                         0.8979   1132131
   macro avg     0.7657    0.9068    0.8111   1132131
weighted avg     0.9346    0.8979    0.9085   1132131



In [ ]:
##  IF  Raw

In [ ]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples.to_numpy())
   
    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 24/24 [19:12<00:00, 48.03s/it]


{'n_estimators': 20, 'con': 0.05}
0.8176793439704795
              precision    recall  f1-score   support

           0     0.9599    0.9494    0.9547    499068
           1     0.6553    0.7082    0.6807     67744

    accuracy                         0.9206    566812
   macro avg     0.8076    0.8288    0.8177    566812
weighted avg     0.9235    0.9206    0.9219    566812



In [ ]:
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples.to_numpy())

    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 24/24 [20:38<00:00, 51.60s/it]


benign only
{'n_estimators': 100}
0.7409370706714709
              precision    recall  f1-score   support

           0     0.9633    0.8755    0.9173    499068
           1     0.4512    0.7539    0.5646     67744

    accuracy                         0.8610    566812
   macro avg     0.7072    0.8147    0.7409    566812
weighted avg     0.9021    0.8610    0.8751    566812

